📘 1️⃣ Project Introduction

# 🏠 AI Interior Design Generator  
This project transforms any room image into a fully redesigned interior using:

- Stable Diffusion v1.5  
- ControlNet Depth Model  
- Multiple LoRA-based interior design styles  
- High-quality enhanced prompts  
- Flask-powered web application  
- Image previews, theme auto-prompts, and loading animations  

The notebook installs dependencies, removes sensitive tokens, 
and launches the full web interface through ngrok.


📘 2️⃣ Install Dependencies

This step installs the required libraries:

- Flask → backend server  
- pyngrok → generate public URL  
- Diffusers → Stable Diffusion pipeline  
- ControlNet → depth-based room structure conditioning  
- controlnet_aux → depth map generation  
- torch + torchvision → GPU inference runtime  
- accelerate + safetensors → optimized loading  

Also creates necessary folder structure:
templates/, static/, uploads/


In [ ]:
!pip install flask pyngrok diffusers transformers accelerate safetensors controlnet_aux torch torchvision
!mkdir -p templates static uploads

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 9.8 MB/s eta 0:00:00


📘 3️⃣ HuggingFace Login

Your notebook originally contained a HuggingFace token.
For security, the token must NOT be hardcoded.

### How to generate your own token:

1. Visit: https://huggingface.co/settings/tokens
2. Click "New Token"
3. Set permission: **Read**
4. Copy the token
5. Replace inside the login() call:

login(token="YOUR_HF_TOKEN_HERE")

⚠️ Never upload your token to GitHub or company websites.


In [ ]:
from huggingface_hub import login
login(token="YOUR_HF_TOKEN_HERE")  # 🔑 replace with your HF token


📘 4️⃣ Model Architecture Overview

This application uses:

### ✔ Stable Diffusion v1.5
Base text-to-image generation model.

### ✔ ControlNet Depth Model
Conditions generation based on:
- room geometry  
- walls  
- furniture edges  
- layout structure  

Ensures the generated design follows your original room shape.

### ✔ LoRA Themes
Optional lightweight fine-tuned models:
- Japanese
- Scandinavian
- Bohemian
- Industrial
- Modern Luxury

If selected, LoRA weights are dynamically loaded into the pipeline.

### ✔ Enhanced Prompts
Each theme comes with a detailed, professional interior design prompt 
for improved realism, lighting, textures, and aesthetic consistency.


📘 5️⃣ Enhanced Design Generation Logic

generate_design() performs:

1. Load uploaded room image  
2. Generate a depth map using MidasDetector (ControlNet Aux)  
3. Load selected LoRA theme (if any)  
4. Use enhanced generation parameters:
   - 50 inference steps
   - guidance_scale = 9.5
   - controlnet_conditioning_scale = 0.8
   - negative prompts for quality filtering  
5. Produce redesigned room image  
6. Save output to /uploads/output.png  

This ensures high-quality, consistent, professional interior renders.


📘 6️⃣ Web Application Overview (Flask)

The Flask app includes 4 routes:

1. "/" → landing page  
2. "/home" → upload + theme selection page  
3. "/generate" → room redesign logic  
4. "/result" → before/after comparison + download button  

User selections are preserved using Flask session variables.

Static images stored in:  
uploads/<filename>

Templates used:
- landing.html  
- home.html  
- result.html  


In [ ]:
%%writefile app.py
from flask import Flask, render_template, request, send_from_directory, redirect, url_for, session
import os, torch
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from controlnet_aux import MidasDetector

app = Flask(__name__)
app.secret_key = "interior_design_secret_key_2024"
app.config['UPLOAD_FOLDER'] = "uploads"
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

# ============================
# Load Models (ControlNet + SD)
# ============================
base_model = "runwayml/stable-diffusion-v1-5"
controlnet_model = "lllyasviel/control_v11f1p_sd15_depth"

print("🔄 Loading ControlNet + Stable Diffusion...")
controlnet = ControlNetModel.from_pretrained(controlnet_model, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    base_model,
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None
)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

depth_detector = MidasDetector.from_pretrained("lllyasviel/ControlNet")
print("✅ Models Loaded Successfully!")

# ============================
# LoRA themes and ENHANCED prompts
# ============================
LORA_THEMES = {
    "None": None,
    "Japanese": "lykon/JapaneseRoom_lora",
    "Scandinavian": "PhilSad/lora-scandinavian-interior",
    "Bohemian": "someuser/boho-room-lora",
    "Industrial": "anotheruser/industrial-loft-lora",
    "Modern Luxury": "modernluxury/interior-lora"
}

# Enhanced detailed prompts for better generation
THEME_PROMPTS = {
    "Japanese": "A stunning Japanese Zen-style interior room featuring authentic tatami floor mats, traditional shoji sliding doors made of wood and rice paper, low wooden chabudai table at center, soft paper lanterns hanging from ceiling, bamboo decorative elements, bonsai tree in corner, natural wood beams, minimalist aesthetic, warm ambient lighting, peaceful atmosphere, high quality, detailed, professional interior photography",

    "Scandinavian": "A bright and airy Scandinavian minimalist interior with pristine white walls, light oak wooden flooring, clean-lined modern furniture in neutral beige and gray tones, large windows with natural light, cozy knitted throw blankets, potted green plants, geometric cushions, simple pendant lights, hygge atmosphere, uncluttered space, professional interior design, high quality, detailed",

    "Bohemian": "A vibrant bohemian chic living room bursting with personality, featuring multiple colorful Persian and Turkish rugs layered on floor, macrame wall hangings, abundant hanging potted plants and trailing vines, mix of ethnic patterned cushions and textiles in warm oranges reds and purples, vintage wooden furniture, rattan chairs, dreamcatchers, floor cushions, warm Edison bulb string lights, eclectic art pieces, cozy maximalist aesthetic, professional interior photography, high detail",

    "Industrial": "An impressive industrial loft interior showcasing exposed red brick walls with visible texture, black metal support beams and pipes across ceiling, distressed reclaimed wood furniture including dining table and shelving, vintage metal factory pendant lights, concrete or dark wood flooring, leather sofa, metal-framed windows, Edison bulb lighting fixtures, urban warehouse aesthetic, masculine design, professional architectural photography, high quality, detailed",

    "Modern Luxury": "An opulent modern luxury penthouse interior featuring polished white marble flooring with gold veining, statement chandelier with crystal elements, elegant contemporary furniture in cream and champagne tones, gold and brass metal accent pieces, floor-to-ceiling windows, plush velvet seating, modern art pieces on walls, indirect LED ambient lighting, luxurious textures, high-end finishes, sophisticated color palette, professional interior design photography, ultra detailed, 8k quality"
}

# ============================
# Generate design with ENHANCED parameters
# ============================
def generate_design(image_path, prompt, theme):
    image = Image.open(image_path).convert("RGB")
    depth_map = depth_detector(image)

    if theme and theme != "None" and LORA_THEMES.get(theme):
        try:
            print(f"🎨 Applying LoRA: {LORA_THEMES[theme]}")
            pipe.unload_lora_weights()
            pipe.load_lora_weights(LORA_THEMES[theme])
            pipe.fuse_lora(lora_scale=0.9)  # Stronger LoRA influence
        except Exception as e:
            print(f"⚠️ Failed to load LoRA for {theme}: {e}")
    else:
        print("🧩 No LoRA selected — using base model.")

    # Enhanced generation parameters for better quality
    output = pipe(
        prompt=prompt,
        negative_prompt="blurry, low quality, distorted, deformed, ugly, bad anatomy, watermark, text, signature, amateur, simple, plain, empty room, incomplete, missing details",
        image=depth_map,
        num_inference_steps=50,  # Increased from 40
        guidance_scale=9.5,  # Increased from 8.5 for stronger prompt adherence
        controlnet_conditioning_scale=0.8,  # Balance between structure and creativity
        generator=torch.manual_seed(42)
    ).images[0]

    output_path = os.path.join(app.config['UPLOAD_FOLDER'], "output.png")
    output.save(output_path)
    return output_path


@app.route("/")
def landing():
    return render_template("landing.html")

@app.route("/home")
def home():
    return render_template("home.html", themes=list(LORA_THEMES.keys()), theme_prompts=THEME_PROMPTS)

@app.route("/generate", methods=["POST"])
def generate():
    uploaded_file = request.files.get("image")
    theme_prompt = request.form.get("prompt")
    theme = request.form.get("theme")

    if uploaded_file and theme_prompt:
        image_path = os.path.join(app.config["UPLOAD_FOLDER"], uploaded_file.filename)
        uploaded_file.save(image_path)

        session['uploaded_image'] = uploaded_file.filename
        session['theme'] = theme
        session['prompt'] = theme_prompt

        result_path = generate_design(image_path, theme_prompt, theme)
        session['output_image'] = "output.png"

        return redirect(url_for('result'))

    return redirect(url_for('home'))

@app.route("/result")
def result():
    output_image = session.get('output_image')
    uploaded_image = session.get('uploaded_image')
    theme = session.get('theme')

    if not output_image:
        return redirect(url_for('home'))

    return render_template("result.html",
                         output_image=output_image,
                         uploaded_image=uploaded_image,
                         theme=theme)

@app.route("/uploads/<path:filename>")
def serve_file(filename):
    return send_from_directory(app.config["UPLOAD_FOLDER"], filename)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)


Writing app.py


📘 7️⃣ UI Template Explanation

### landing.html
- Hero banner  
- Feature cards  
- Theme showcase  
- "Start Designing" button  

### home.html
- Upload photo  
- Theme dropdown  
- Auto-fill prompt feature  
- Live preview  
- Loading screen while AI renders  

### result.html
- Before / After comparison  
- Download button  
- Tips for better results  
- Animated glow effects  


In [ ]:
%%writefile templates/landing.html
<!DOCTYPE html>
<html>
<head>
    <title>🏠 Interior Design Generator - AI Powered</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="landing-container">
        <div class="hero-section fade-in">
            <h1 class="landing-title">🏠 AI Interior Design Studio</h1>
            <p class="landing-subtitle">Transform Your Space with Artificial Intelligence</p>

            <div class="features-grid">
                <div class="feature-card slide-in-left">
                    <span class="feature-icon">📸</span>
                    <h3>Upload Your Room</h3>
                    <p>Take a photo of any room in your home</p>
                </div>

                <div class="feature-card slide-in-up">
                    <span class="feature-icon">🎨</span>
                    <h3>Choose a Style</h3>
                    <p>Select from 5+ professional design themes</p>
                </div>

                <div class="feature-card slide-in-right">
                    <span class="feature-icon">✨</span>
                    <h3>Get Your Design</h3>
                    <p>AI generates stunning redesigns in seconds</p>
                </div>
            </div>

            <div class="themes-showcase">
                <h2 class="showcase-title">Available Design Themes</h2>
                <div class="theme-tags">
                    <span class="theme-tag">🇯🇵 Japanese Zen</span>
                    <span class="theme-tag">🌿 Scandinavian</span>
                    <span class="theme-tag">🎭 Bohemian Chic</span>
                    <span class="theme-tag">🏭 Industrial Loft</span>
                    <span class="theme-tag">💎 Modern Luxury</span>
                </div>
            </div>

            <a href="{{ url_for('home') }}" class="start-btn pulse">
                🚀 Start Designing Now
            </a>

            <div class="tech-info">
                <p>🤖 Powered by ControlNet + Stable Diffusion + LoRA Technology</p>
            </div>
        </div>
    </div>
</body>
</html>


Writing templates/landing.html


In [ ]:
%%writefile templates/home.html
<!DOCTYPE html>
<html>
<head>
    <title>🏠 Create Your Design</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
    <script>
        function previewImage(event) {
            const file = event.target.files[0];
            if (file) {
                const reader = new FileReader();
                reader.onload = function() {
                    const preview = document.getElementById('preview');
                    const placeholder = document.getElementById('placeholder');
                    preview.src = reader.result;
                    preview.style.display = 'block';
                    placeholder.style.display = 'none';

                    // Show success message
                    const uploadLabel = document.querySelector('.file-label');
                    uploadLabel.innerHTML = '✅ Image Uploaded Successfully!';
                    uploadLabel.style.background = 'linear-gradient(90deg, #00897b, #00bfa5)';
                };
                reader.readAsDataURL(file);
            }
        }

        function showLoading() {
            document.getElementById('loading').style.display = 'flex';
        }

        function autofillPrompt() {
            const theme = document.getElementById("theme").value;
            const promptBox = document.getElementById("prompt");
            const prompts = {{ theme_prompts|tojson }};

            if (prompts[theme]) {
                promptBox.value = prompts[theme];
                // Show notification
                showNotification('✨ Prompt loaded for ' + theme + ' style!');
            } else {
                promptBox.value = "";
                promptBox.placeholder = "Describe your desired interior style in detail...";
            }
        }

        function showNotification(message) {
            const notification = document.createElement('div');
            notification.className = 'notification';
            notification.textContent = message;
            document.body.appendChild(notification);

            setTimeout(() => {
                notification.classList.add('show');
            }, 100);

            setTimeout(() => {
                notification.classList.remove('show');
                setTimeout(() => notification.remove(), 300);
            }, 3000);
        }

        function validateForm(event) {
            const fileInput = document.getElementById('imageInput');
            const promptInput = document.getElementById('prompt');

            if (!fileInput.files.length) {
                event.preventDefault();
                showNotification('⚠️ Please upload a room image first!');
                return false;
            }

            if (!promptInput.value.trim()) {
                event.preventDefault();
                showNotification('⚠️ Please enter or select a design prompt!');
                return false;
            }

            showLoading();
            return true;
        }
    </script>
</head>
<body>
    <div id="loading">
        <div class="spinner"></div>
        <p>✨ AI is generating your stunning interior design...</p>
        <p class="loading-subtext">This may take 30-60 seconds. Please don't close this page!</p>
    </div>

    <div class="container fade-in">
        <a href="{{ url_for('landing') }}" class="back-btn">← Back to Home</a>

        <h1 class="title">🎨 Create Your Design</h1>
        <p class="subtitle">Upload your room photo and let AI transform it!</p>

        <div class="steps-guide">
            <div class="step">
                <span class="step-number">1</span>
                <p>Upload your room photo</p>
            </div>
            <div class="step-arrow">→</div>
            <div class="step">
                <span class="step-number">2</span>
                <p>Select a design theme</p>
            </div>
            <div class="step-arrow">→</div>
            <div class="step">
                <span class="step-number">3</span>
                <p>Customize prompt (optional)</p>
            </div>
            <div class="step-arrow">→</div>
            <div class="step">
                <span class="step-number">4</span>
                <p>Generate & download!</p>
            </div>
        </div>

        <form method="post" action="{{ url_for('generate') }}" enctype="multipart/form-data" class="upload-form" onsubmit="return validateForm(event)">

            <div class="upload-section">
                <label class="file-label" for="imageInput" onclick="document.getElementById('imageInput').click()">
                    📸 Click to Upload Room Image
                </label>
                <input id="imageInput" type="file" name="image" accept="image/*" onchange="previewImage(event)" hidden>

                <div class="preview-container">
                    <div id="placeholder" class="image-placeholder">
                        <span class="placeholder-icon">🖼️</span>
                        <p>Your image will appear here</p>
                    </div>
                    <img id="preview" style="display:none;">
                </div>
            </div>

            <div class="theme-section">
                <h3 class="section-heading">
                    <span class="heading-icon">🎭</span>
                    Choose Your Interior Theme
                </h3>
                <select id="theme" name="theme" class="theme-dropdown" onchange="autofillPrompt()" required>
                    <option value="" disabled selected>Select a theme...</option>
                    {% for theme in themes %}
                        <option value="{{ theme }}">{{ theme }}</option>
                    {% endfor %}
                </select>
                <p class="helper-text">💡 Selecting a theme will auto-fill a professional prompt below</p>
            </div>

            <div class="prompt-section">
                <h3 class="section-heading">
                    <span class="heading-icon">✍️</span>
                    Design Prompt (Customizable)
                </h3>
                <textarea id="prompt" name="prompt" class="prompt-box" rows="5" placeholder="Select a theme above, or write your own detailed description of the desired interior style..."></textarea>
                <p class="helper-text">💡 Be detailed! Mention colors, furniture, lighting, decorations, and atmosphere for best results</p>
            </div>

            <button type="submit" class="generate-btn pulse">
                <span class="btn-icon">🚀</span>
                Generate My Design
                <span class="btn-icon">✨</span>
            </button>
        </form>
    </div>

    <footer>
        <div class="footer-content">
            <p class="footer-main">🌍 Powered by ControlNet + Stable Diffusion + LoRA</p>
            <p class="footer-sub">AI Design Studio | Professional Interior Visualization</p>
        </div>
    </footer>
</body>
</html>

Writing templates/home.html


In [ ]:
%%writefile templates/result.html
<!DOCTYPE html>
<html>
<head>
    <title>✨ Your Design is Ready!</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
    <script>
        function showNotification(message) {
            const notification = document.createElement('div');
            notification.className = 'notification';
            notification.textContent = message;
            document.body.appendChild(notification);

            setTimeout(() => {
                notification.classList.add('show');
            }, 100);

            setTimeout(() => {
                notification.classList.remove('show');
                setTimeout(() => notification.remove(), 300);
            }, 3000);
        }

        function downloadImage() {
            showNotification('💾 Your design is downloading...');
        }
    </script>
</head>
<body>
    <div class="container fade-in">
        <a href="{{ url_for('home') }}" class="back-btn">← Create Another Design</a>

        <div class="success-banner">
            <span class="success-icon">🎉</span>
            <h1 class="success-title">Your Design is Ready!</h1>
            <p class="success-subtitle">AI has transformed your space with {{ theme }} style</p>
        </div>

        <div class="comparison-section">
            <div class="comparison-container">
                <div class="image-box">
                    <h3 class="image-label">
                        <span class="label-icon">📷</span>
                        Original Room
                    </h3>
                    <div class="image-wrapper">
                        <img src="{{ url_for('serve_file', filename=uploaded_image) }}" alt="Original Room" class="comparison-img">
                    </div>
                </div>

                <div class="vs-divider">
                    <span class="vs-text">→</span>
                </div>

                <div class="image-box">
                    <h3 class="image-label">
                        <span class="label-icon">✨</span>
                        AI Redesigned ({{ theme }})
                    </h3>
                    <div class="image-wrapper glow-effect">
                        <img src="{{ url_for('serve_file', filename=output_image) }}" alt="Redesigned Room" class="comparison-img result-glow">
                    </div>
                </div>
            </div>
        </div>

        <div class="action-section">
            <h2 class="action-heading">What would you like to do?</h2>

            <div class="action-buttons">
                <a href="{{ url_for('serve_file', filename=output_image) }}" download="ai_interior_design.png" class="action-btn download-btn" onclick="downloadImage()">
                    <span class="btn-icon">💾</span>
                    Download Design
                </a>

                <a href="{{ url_for('home') }}" class="action-btn new-design-btn">
                    <span class="btn-icon">🎨</span>
                    Create New Design
                </a>

                <a href="{{ url_for('landing') }}" class="action-btn home-btn">
                    <span class="btn-icon">🏠</span>
                    Back to Home
                </a>
            </div>
        </div>

        <div class="tips-section">
            <h3 class="tips-heading">💡 Tips for Better Results</h3>
            <div class="tips-grid">
                <div class="tip-card">
                    <span class="tip-icon">📸</span>
                    <p>Use well-lit photos with clear room structure</p>
                </div>
                <div class="tip-card">
                    <span class="tip-icon">🎯</span>
                    <p>Select the theme that matches your vision</p>
                </div>
                <div class="tip-card">
                    <span class="tip-icon">✍️</span>
                    <p>Add specific details to the prompt for precision</p>
                </div>
                <div class="tip-card">
                    <span class="tip-icon">🔄</span>
                    <p>Try different themes on the same room!</p>
                </div>
            </div>
        </div>
    </div>

    <footer>
        <div class="footer-content">
            <p class="footer-main">🌍 Powered by ControlNet + Stable Diffusion + LoRA</p>
            <p class="footer-sub">AI Design Studio | Professional Interior Visualization</p>
        </div>
    </footer>
</body>
</html>

Writing templates/result.html


📘 8️⃣ CSS Overview (Modern UI)

The CSS provides:

- Gradient animated background  
- Glassmorphism effects  
- Smooth hover & fade animations  
- Pulsing buttons  
- Loading spinner overlay  
- Notification popups  
- Responsive layout for mobile  

Matches the professional aesthetic expected from 
an interior design application.


In [ ]:
%%writefile static/style.css
@import url('https://fonts.googleapis.com/css2?family=Poppins:wght@400;600;700&display=swap');

* {
    margin: 0;
    padding: 0;
    box-sizing: border-box;
}

body {
    font-family: 'Poppins', sans-serif;
    background: linear-gradient(135deg, #0d1117, #161b22, #1a1f2e);
    background-size: 400% 400%;
    animation: gradientShift 15s ease infinite;
    color: #e0e0e0;
    min-height: 100vh;
    display: flex;
    flex-direction: column;
    align-items: center;
    padding: 20px;
}

@keyframes gradientShift {
    0%, 100% { background-position: 0% 50%; }
    50% { background-position: 100% 50%; }
}

/* ========== LANDING PAGE ========== */
.landing-container {
    width: 100%;
    max-width: 1200px;
    display: flex;
    justify-content: center;
    align-items: center;
    min-height: 90vh;
}

.hero-section {
    text-align: center;
    padding: 40px;
}

.landing-title {
    font-size: 3.5rem;
    color: #00bfa5;
    margin-bottom: 15px;
    text-shadow: 0 0 20px rgba(0, 191, 165, 0.5);
    animation: titleFloat 3s ease-in-out infinite;
}

@keyframes titleFloat {
    0%, 100% { transform: translateY(0px); }
    50% { transform: translateY(-10px); }
}

.landing-subtitle {
    font-size: 1.3rem;
    color: #b0b0b0;
    margin-bottom: 50px;
}

.features-grid {
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
    gap: 30px;
    margin-bottom: 50px;
}

.feature-card {
    background: rgba(30, 40, 50, 0.6);
    backdrop-filter: blur(10px);
    border-radius: 15px;
    padding: 30px;
    border: 1px solid rgba(0, 191, 165, 0.2);
    transition: all 0.3s ease;
}

.feature-card:hover {
    transform: translateY(-10px);
    border-color: rgba(0, 191, 165, 0.6);
    box-shadow: 0 10px 30px rgba(0, 191, 165, 0.3);
}

.feature-icon {
    font-size: 3rem;
    display: block;
    margin-bottom: 15px;
}

.feature-card h3 {
    color: #00bfa5;
    margin-bottom: 10px;
    font-size: 1.2rem;
}

.feature-card p {
    color: #b0b0b0;
    font-size: 0.95rem;
}

.themes-showcase {
    margin: 50px 0;
}

.showcase-title {
    color: #00bfa5;
    font-size: 1.8rem;
    margin-bottom: 25px;
}

.theme-tags {
    display: flex;
    flex-wrap: wrap;
    gap: 15px;
    justify-content: center;
}

.theme-tag {
    background: rgba(0, 191, 165, 0.15);
    border: 2px solid rgba(0, 191, 165, 0.4);
    padding: 12px 24px;
    border-radius: 25px;
    color: #00bfa5;
    font-weight: 600;
    transition: all 0.3s ease;
    cursor: default;
}

.theme-tag:hover {
    background: rgba(0, 191, 165, 0.25);
    border-color: rgba(0, 191, 165, 0.7);
    transform: scale(1.05);
}

.start-btn {
    display: inline-block;
    margin-top: 40px;
    padding: 18px 50px;
    background: linear-gradient(90deg, #00695c, #00bfa5);
    color: white;
    text-decoration: none;
    border-radius: 50px;
    font-size: 1.3rem;
    font-weight: 600;
    transition: all 0.3s ease;
    box-shadow: 0 5px 20px rgba(0, 191, 165, 0.4);
}

.start-btn:hover {
    transform: scale(1.08);
    box-shadow: 0 8px 30px rgba(0, 191, 165, 0.6);
}

.tech-info {
    margin-top: 50px;
    color: #888;
    font-size: 0.9rem;
}

/* ========== MAIN CONTAINER ========== */
.container {
    width: 90%;
    max-width: 900px;
    background: rgba(30, 40, 50, 0.85);
    backdrop-filter: blur(15px);
    border-radius: 20px;
    padding: 40px;
    margin: 20px 0;
    box-shadow: 0 10px 40px rgba(0, 0, 0, 0.5);
    border: 1px solid rgba(0, 191, 165, 0.1);
}

.back-btn {
    display: inline-block;
    margin-bottom: 20px;
    padding: 10px 20px;
    background: rgba(0, 191, 165, 0.15);
    color: #00bfa5;
    text-decoration: none;
    border-radius: 10px;
    border: 1px solid rgba(0, 191, 165, 0.3);
    transition: all 0.3s ease;
    font-size: 0.95rem;
}

.back-btn:hover {
    background: rgba(0, 191, 165, 0.25);
    transform: translateX(-5px);
}

.title {
    font-size: 2.5rem;
    color: #00bfa5;
    margin-bottom: 10px;
    text-align: center;
    text-shadow: 0 0 15px rgba(0, 191, 165, 0.5);
}

.subtitle {
    color: #b0b0b0;
    font-size: 1.1rem;
    margin-bottom: 30px;
    text-align: center;
}

/* ========== STEPS GUIDE ========== */
.steps-guide {
    display: flex;
    justify-content: center;
    align-items: center;
    flex-wrap: wrap;
    gap: 15px;
    margin: 30px 0;
    padding: 20px;
    background: rgba(0, 191, 165, 0.05);
    border-radius: 15px;
    border: 1px dashed rgba(0, 191, 165, 0.3);
}

.step {
    display: flex;
    flex-direction: column;
    align-items: center;
    text-align: center;
}

.step-number {
    width: 40px;
    height: 40px;
    background: linear-gradient(135deg, #00695c, #00bfa5);
    color: white;
    border-radius: 50%;
    display: flex;
    align-items: center;
    justify-content: center;
    font-weight: 700;
    font-size: 1.1rem;
    margin-bottom: 8px;
    box-shadow: 0 3px 10px rgba(0, 191, 165, 0.4);
}

.step p {
    font-size: 0.85rem;
    color: #b0b0b0;
    max-width: 100px;
}

.step-arrow {
    color: #00bfa5;
    font-size: 1.5rem;
    font-weight: bold;
}

/* ========== FORM SECTIONS ========== */
.upload-form {
    display: flex;
    flex-direction: column;
    gap: 30px;
    margin-top: 30px;
}

.upload-section, .theme-section, .prompt-section {
    display: flex;
    flex-direction: column;
    gap: 15px;
}

.section-heading {
    font-size: 1.2rem;
    color: #00bfa5;
    display: flex;
    align-items: center;
    gap: 10px;
    margin-bottom: 10px;
}

.heading-icon {
    font-size: 1.4rem;
}

.helper-text {
    font-size: 0.85rem;
    color: #888;
    font-style: italic;
    margin-top: -8px;
}

/* ========== FILE UPLOAD ========== */
.file-label {
    background: linear-gradient(90deg, #00796b, #009688);
    color: #fff;
    font-weight: 600;
    padding: 16px 24px;
    border-radius: 12px;
    cursor: pointer;
    text-align: center;
    transition: all 0.3s ease;
    box-shadow: 0 4px 15px rgba(0, 150, 136, 0.3);
}

.file-label:hover {
    transform: scale(1.03);
    box-shadow: 0 6px 20px rgba(0, 150, 136, 0.5);
}

.preview-container {
    width: 100%;
    min-height: 300px;
    border-radius: 12px;
    overflow: hidden;
    background: rgba(0, 0, 0, 0.3);
    border: 2px dashed rgba(0, 191, 165, 0.3);
    display: flex;
    align-items: center;
    justify-content: center;
}

.image-placeholder {
    text-align: center;
    color: #666;
}

.placeholder-icon {
    font-size: 4rem;
    display: block;
    margin-bottom: 15px;
    opacity: 0.5;
}

#preview {
    width: 100%;
    height: auto;
    object-fit: contain;
    animation: imageReveal 0.5s ease;
}

@keyframes imageReveal {
    from { opacity: 0; transform: scale(0.95); }
    to { opacity: 1; transform: scale(1); }
}

/* ========== THEME DROPDOWN ========== */
.theme-dropdown {
    width: 100%;
    padding: 14px;
    border-radius: 12px;
    background: rgba(255, 255, 255, 0.08);
    color: #e0e0e0;
    border: 1px solid rgba(0, 191, 165, 0.3);
    font-size: 1rem;
    outline: none;
    cursor: pointer;
    transition: all 0.3s ease;
}

.theme-dropdown:hover, .theme-dropdown:focus {
    border-color: rgba(0, 191, 165, 0.6);
    background: rgba(255, 255, 255, 0.12);
}

.theme-dropdown option {
    background: #1a1f2e;
    color: #e0e0e0;
    padding: 10px;
}

/* ========== PROMPT BOX ========== */
.prompt-box {
    width: 100%;
    padding: 14px;
    border-radius: 12px;
    border: 1px solid rgba(0, 191, 165, 0.3);
    background: rgba(255, 255, 255, 0.05);
    color: #e0e0e0;
    font-size: 0.95rem;
    outline: none;
    font-family: 'Poppins', sans-serif;
    resize: vertical;
    transition: all 0.3s ease;
}

.prompt-box:focus {
    border-color: rgba(0, 191, 165, 0.6);
    background: rgba(255, 255, 255, 0.08);
    box-shadow: 0 0 15px rgba(0, 191, 165, 0.2);
}

/* ========== BUTTONS ========== */
.generate-btn {
    width: 100%;
    background: linear-gradient(90deg, #00695c, #00bfa5);
    color: white;
    border: none;
    border-radius: 15px;
    padding: 18px;
    font-size: 1.2rem;
    font-weight: 600;
    cursor: pointer;
    transition: all 0.3s ease;
    box-shadow: 0 5px 20px rgba(0, 191, 165, 0.4);
    display: flex;
    align-items: center;
    justify-content: center;
    gap: 10px;
}

.generate-btn:hover {
    transform: scale(1.03);
    box-shadow: 0 8px 30px rgba(0, 191, 165, 0.6);
}

.btn-icon {
    font-size: 1.3rem;
}

/* ========== RESULT PAGE ========== */
.success-banner {
    text-align: center;
    padding: 30px;
    background: rgba(0, 191, 165, 0.1);
    border-radius: 15px;
    margin-bottom: 40px;
    border: 2px solid rgba(0, 191, 165, 0.3);
}

.success-icon {
    font-size: 4rem;
    display: block;
    margin-bottom: 15px;
    animation: bounce 1s ease infinite;
}

@keyframes bounce {
    0%, 100% { transform: translateY(0); }
    50% { transform: translateY(-15px); }
}

.success-title {
    font-size: 2.2rem;
    color: #00bfa5;
    margin-bottom: 10px;
}

.success-subtitle {
    color: #b0b0b0;
    font-size: 1.1rem;
}

.comparison-section {
    margin: 40px 0;
}

.comparison-container {
    display: grid;
    grid-template-columns: 1fr auto 1fr;
    gap: 20px;
    align-items: center;
}

.image-box {
    background: rgba(0, 0, 0, 0.3);
    border-radius: 15px;
    padding: 20px;
    border: 1px solid rgba(0, 191, 165, 0.2);
}

.image-label {
    color: #00bfa5;
    margin-bottom: 15px;
    display: flex;
    align-items: center;
    gap: 8px;
    font-size: 1.1rem;
}

.label-icon {
    font-size: 1.3rem;
}

.image-wrapper {
    border-radius: 12px;
    overflow: hidden;
    background: #000;
}

.comparison-img {
    width: 100%;
    height: auto;
    display: block;
}

.glow-effect {
    animation: glowPulse 2s ease-in-out infinite;
}

@keyframes glowPulse {
    0%, 100% { box-shadow: 0 0 20px rgba(0, 191, 165, 0.3); }
    50% { box-shadow: 0 0 35px rgba(0, 191, 165, 0.6); }
}

.vs-divider {
    font-size: 2rem;
    color: #00bfa5;
    font-weight: bold;
}

.vs-text {
    display: block;
    animation: slideArrow 1.5s ease-in-out infinite;
}

@keyframes slideArrow {
    0%, 100% { transform: translateX(0); }
    50% { transform: translateX(10px); }
}

/* ========== ACTION SECTION ========== */
.action-section {
    margin: 50px 0;
    text-align: center;
}

.action-heading {
    color: #00bfa5;
    font-size: 1.6rem;
    margin-bottom: 25px;
}

.action-buttons {
    display: flex;
    flex-wrap: wrap;
    gap: 15px;
    justify-content: center;
}

.action-btn {
    flex: 1;
    min-width: 200px;
    padding: 15px 25px;
    border-radius: 12px;
    text-decoration: none;
    font-weight: 600;
    transition: all 0.3s ease;
    display: flex;
    align-items: center;
    justify-content: center;
    gap: 10px;
    font-size: 1rem;
}

.download-btn {
    background: linear-gradient(90deg, #00796b, #00bfa5);
    color: white;
    box-shadow: 0 4px 15px rgba(0, 191, 165, 0.3);
}

.download-btn:hover {
    transform: scale(1.05);
    box-shadow: 0 6px 25px rgba(0, 191, 165, 0.5);
}

.new-design-btn {
    background: rgba(0, 121, 107, 0.3);
    color: #00bfa5;
    border: 2px solid rgba(0, 191, 165, 0.5);
}

.new-design-btn:hover {
    background: rgba(0, 121, 107, 0.5);
    transform: scale(1.05);
}

.home-btn {
    background: rgba(100, 100, 100, 0.3);
    color: #b0b0b0;
    border: 2px solid rgba(150, 150, 150, 0.4);
}

.home-btn:hover {
    background: rgba(100, 100, 100, 0.5);
    color: #e0e0e0;
    transform: scale(1.05);
}

/* ========== TIPS SECTION ========== */
.tips-section {
    margin-top: 50px;
    padding: 30px;
    background: rgba(0, 191, 165, 0.05);
    border-radius: 15px;
    border: 1px dashed rgba(0, 191, 165, 0.3);
}

.tips-heading {
    color: #00bfa5;
    text-align: center;
    margin-bottom: 25px;
    font-size: 1.4rem;
}

.tips-grid {
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
    gap: 20px;
}

.tip-card {
    background: rgba(30, 40, 50, 0.5);
    padding: 20px;
    border-radius: 12px;
    text-align: center;
    border: 1px solid rgba(0, 191, 165, 0.2);
    transition: all 0.3s ease;
}

.tip-card:hover {
    transform: translateY(-5px);
    border-color: rgba(0, 191, 165, 0.5);
}

.tip-icon {
    font-size: 2rem;
    display: block;
    margin-bottom: 10px;
}

.tip-card p {
    color: #b0b0b0;
    font-size: 0.9rem;
}

/* ========== LOADING ANIMATION ========== */
#loading {
    position: fixed;
    display: none;
    justify-content: center;
    align-items: center;
    flex-direction: column;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    background: rgba(0, 0, 0, 0.92);
    z-index: 9999;
    color: #00bfa5;
}

#loading p {
    font-size: 1.3rem;
    margin-top: 20px;
    text-align: center;
}

.loading-subtext {
    font-size: 1rem !important;
    color: #888 !important;
    margin-top: 10px !important;
}

.spinner {
    border: 6px solid rgba(0, 191, 165, 0.1);
    border-top: 6px solid #00bfa5;
    border-radius: 50%;
    width: 70px;
    height: 70px;
    animation: spin 1s linear infinite;
}

@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}

/* ========== NOTIFICATION ========== */
.notification {
    position: fixed;
    top: 20px;
    right: 20px;
    background: rgba(0, 191, 165, 0.95);
    color: white;
    padding: 15px 25px;
    border-radius: 10px;
    box-shadow: 0 5px 20px rgba(0, 0, 0, 0.5);
    z-index: 10000;
    opacity: 0;
    transform: translateX(400px);
    transition: all 0.3s ease;
    font-weight: 600;
}

.notification.show {
    opacity: 1;
    transform: translateX(0);
}

/* ========== FOOTER ========== */
footer {
    width: 100%;
    margin-top: auto;
    padding: 30px 20px;
    text-align: center;
}

.footer-content {
    max-width: 900px;
    margin: 0 auto;
}

.footer-main {
    font-size: 0.95rem;
    color: #00bfa5;
    margin-bottom: 8px;
    font-weight: 600;
}

.footer-sub {
    font-size: 0.85rem;
    color: #888;
}

/* ========== ANIMATIONS ========== */
.fade-in {
    animation: fadeIn 0.8s ease;
}

@keyframes fadeIn {
    from { opacity: 0; }
    to { opacity: 1; }
}

.slide-in-left {
    animation: slideInLeft 0.6s ease;
}

@keyframes slideInLeft {
    from {
        opacity: 0;
        transform: translateX(-50px);
    }
    to {
        opacity: 1;
        transform: translateX(0);
    }
}

.slide-in-right {
    animation: slideInRight 0.6s ease;
}

@keyframes slideInRight {
    from {
        opacity: 0;
        transform: translateX(50px);
    }
    to {
        opacity: 1;
        transform: translateX(0);
    }
}

.slide-in-up {
    animation: slideInUp 0.6s ease;
}

@keyframes slideInUp {
    from {
        opacity: 0;
        transform: translateY(50px);
    }
    to {
        opacity: 1;
        transform: translateY(0);
    }
}

.pulse {
    animation: pulse 2s ease-in-out infinite;
}

@keyframes pulse {
    0%, 100% {
        transform: scale(1);
    }
    50% {
        transform: scale(1.05);
    }
}

/* ========== RESPONSIVE ========== */
@media (max-width: 768px) {
    .landing-title {
        font-size: 2.5rem;
    }

    .comparison-container {
        grid-template-columns: 1fr;
    }

    .vs-divider {
        transform: rotate(90deg);
        margin: 10px 0;
    }

    .steps-guide {
        flex-direction: column;
    }

    .step-arrow {
        transform: rotate(90deg);
    }

    .action-buttons {
        flex-direction: column;
    }

    .action-btn {
        min-width: 100%;
    }
}



Writing static/style.css


📘 9️⃣ Killing Existing Processes (Flask + ngrok)

Before starting the new server, terminate any leftover sessions:

!pkill -f flask || echo "No Flask process running"
!pkill -f ngrok || echo "No ngrok process running"

This prevents:
- Port conflicts (8000 already in use)
- Duplicate Flask instances
- Ngrok tunnel failures



In [ ]:
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"


^C
^C


📘 🔟 Check Which Process Is Using Port 8000

Run this to identify the active process:

!lsof -i :8000

Example output:
python3   599   user   ...   TCP *:8000 (LISTEN)

Here, "599" is the PID using port 8000.


In [ ]:
!lsof -i :8000

📘 1️⃣1️⃣ Kill a Specific PID (If Needed)

If port 8000 is occupied, kill the process:

!kill -9 599

After this, port 8000 becomes free
and the Flask server can start safely.


In [ ]:
!kill -9 599

/bin/bash: line 1: kill: (599) - No such process


📘 1️⃣2️⃣ Start Flask Server in Background

Run Flask without blocking the notebook:

!nohup python app.py > flask.log 2>&1 &

Notes:
- nohup keeps Flask running in the background
- Output goes into flask.log
- Notebook remains responsive


In [ ]:
!nohup python app.py > flask.log 2>&1 &


📘 1️⃣3️⃣ Start Ngrok Tunnel (Public URL)

Ngrok exposes your local Flask app on the internet.

Steps:

1. Create account  
   https://dashboard.ngrok.com/signup  

2. Find your auth token  
   https://dashboard.ngrok.com/get-started/your-authtoken  

3. Add your token:

from pyngrok import ngrok, conf
conf.get_default().auth_token = "YOUR_NGROK_TOKEN"

4. Start the public tunnel:

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)


In [ ]:
from pyngrok import ngrok, conf
conf.get_default().auth_token = "2ztgrogNy3iJnxaz8wchWgCw1X7_4vgQZE6DfL2kHMu2DmQzM"  # 🔑 replace with your ngrok token

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

!sleep 3 && tail -n 30 flask.log


🌍 Public URL: NgrokTunnel: "https://f8f0e76ebe8f.ngrok-free.app" -> "http://localhost:8000"


📘 1️⃣4️⃣ View Flask Logs (Debugging)

To view last 30 lines of the log:

!tail -n 30 flask.log

To view last 50 lines:

!tail -n 50 flask.log


In [ ]:
!tail -n 50 flask.log

🔄 Loading ControlNet + Stable Diffusion...
Loading pipeline components...: 100%|██████████| 6/6 [00:18<00:00,  3.13s/it]
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(
✅ Models Loaded Succes

In [ ]:
📘 1️⃣5️⃣ Project Ready 🎉

Your AI Interior Design Generator is fully functional!

Users can now:

✔ Upload a room photo  
✔ Select interior style themes  
✔ Auto-load professional prompts  
✔ Generate redesigned interior images  
✔ Compare Before/After results  
✔ Download final output  
✔ Create unlimited new designs  

The system is powered by:
- Stable Diffusion v1.5  
- ControlNet (Depth)  
- Optional LoRA themes  
- GPU-accelerated inference  
- Fully responsive Flask UI  


In [ ]:
📘 1️⃣6️⃣ Helpful Resources

Stable Diffusion v1.5 Model:
https://huggingface.co/runwayml/stable-diffusion-v1-5

ControlNet Documentation:
https://huggingface.co/lllyasviel

Diffusers Library:
https://huggingface.co/docs/diffusers

LoRA Training Guide:
https://huggingface.co/docs/diffusers/main/en/training/lora

pyngrok Documentation:
https://pyngrok.readthedocs.io/en/latest/

Flask Framework:
https://flask.palletsprojects.com/
